# Damavis Challenge

Consider a rectangular board consisting of **n × m cells**. There is a snake lying on some of the
cells, and all of the other cells are empty. **The snake consists of one or more cells** such that
cells with consecutive numbers are either horizontally or vertically adjacent. **The first cell is
the snake's head, and the last cell is the snake's tail**.


On each turn, the snake's head moves to one of the horizontally or vertically adjacent cells,
the second cell of the snake moves to the cell where the head was situated, the third cell
takes the former place of the second cell, etc. **All these movements happen simultaneously**,
so the head could potentially take the place of the tail. There are two configurations of the
snake's cells that are prohibited: self-intersection (meaning that after each step any pair of
snake cells should have pairwise different coordinates), and crossing the board's border. **The
path is a sequence of characters L, R, D, and U** (corresponding to left, right, down, and up,
respectively) describing the movements of snake's head on each turn. **How many distinct
valid paths of length p can the snake make on the board?**

**EXAMPLE:**

![Snake Example](snake_example.png "Snake Example")

For board = `[4, 3]`, snake = `[[2, 2], [3, 2], [3, 1], [3, 0], [2, 0], [1, 0], [0, 0]]`, and `depth = 3`, the
output should be `numberOfAvailableDifferentPaths(board, snake, depth) = 7`.

Here are all of the valid paths with a length of 3 that the snake can make:
- LLU
- LUR
- LUL
- ULL
- ULD
- LUU
- ULU

**CONTRACT INPUT OUTPUT**:

There are **3 input argument** in this challenge:

- `board` as array.integer:

    The board is an array describing the dimensions of the board. **board[0] stands for the number of rows, and board[1] corresponds to the number of columns**. Guaranteed constraints:

        board.length = 2,
        1 ≤ board[i] ≤ 10.
        
- `snake` as array.array.integer:
    
    The snake is an array that describes the configuration of the snake's body on the board. **snake[0] corresponds to the initial coordinates of the snake's head**, snake[1] corresponds to coordinates of the second cell, etc. It is guaranteed that **snake[i] and snake[i + 1] are horizontally or vertically adjacent**, and that its **initial configuration is valid** (i.e. there are no self-intersections and the snake's entire body lies inside the board). Guaranteed constraints:
    
        3 ≤ snake.length ≤ 7,
        snake[i].length = 2,
        0 ≤ snake[i][j] < board[j].
        
- `depth` as integer:

    The depth is the paths depth. You have to discard all paths with a different path length. Guaranteed constraints:
    
        1 ≤ n ≤ 20.
        
And the **output**:

- `output` as integer:

    The number of distinct valid paths of length p that the snake can make, modulo $10^9 + 7$.

## Variable declaration

In [1]:
HORIZONTAL_INDEX = 0
VERTICAL_INDEX = 1

LEFT = 0
RIGHT = 1
DOWN = 2
UP = 3

DIRECTIONS_LETTERS = 'LRDU'
DIRECTIONS_MAP = {
    'L': LEFT,
    'R': RIGHT,
    'D': DOWN,
    'U': UP
}

In [2]:
board = [4,3]
snake = [
    [2, 2],
    [3, 2],
    [3, 1],
    [3, 0],
    [2, 0],
    [1, 0],
    [0, 0]
]
depth = 3

## Auxiliar functions

We will do a function to print the snake in the board so the paths are more visual and easy to understand

In [3]:
HEAD_SYMBOL = ':' # If it is the snake's head print ':'
BODY_SYMBOL = '=' # If it is the body, print '='
EMPTY_SYMBOL = ' ' # If the cell is empty, print ' '

def print_board(board : list, snake : list):
    """Print the current state of the snake in the board

    Parameters
    ----------
    board : list
        A list of length 2 with the dimensions of the board.
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    """

    for j in range(board[VERTICAL_INDEX]):
        for i in range(board[HORIZONTAL_INDEX]):
            if i == 0:
                print('| ', end='')
            if [i,j] in snake: # If the snake is in this cell print it
                if [i,j] == snake[0]:
                    print(f'{HEAD_SYMBOL} | ', end='')
                else:
                    print(f'{BODY_SYMBOL} | ', end='')
            else:
                print(f'{EMPTY_SYMBOL} | ', end='')
        print()
    print()

In [4]:
print_board(board, snake)

| = | = | = | = | 
|   |   |   | = | 
|   |   | : | = | 



In [5]:
def move_snake(snake : list, direction : int):
    """Creates a new snake in the given direction.

    Parameters
    ----------
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    direction : int
        The direction to move the snake -> 0, 1, 2, 3 = LEFT, RIGHT, DOWN, UP
    
    
    Returns
    -------
    list
        A list of cells with the new snake
    """
    
    snake_copy = snake.copy()
    # Get the snake's head
    snakes_head = snake_copy[0]
    # Remove the tail of the snake
    snake_copy.pop()
    
    # Get the new head with the movement
    new_head = snakes_head.copy()
    if direction == LEFT:
        new_head[HORIZONTAL_INDEX] = new_head[HORIZONTAL_INDEX] - 1
    elif direction == RIGHT:
        new_head[HORIZONTAL_INDEX] = new_head[HORIZONTAL_INDEX] + 1
    elif direction == DOWN:
        new_head[VERTICAL_INDEX] = new_head[VERTICAL_INDEX] + 1
    elif direction == UP:
        new_head[VERTICAL_INDEX] = new_head[VERTICAL_INDEX] - 1
        
    # Insert the new head in snake's body
    snake_copy.insert(0, new_head)
    
    return snake_copy

In [6]:
print_board(board, snake)
print_board(board, move_snake(snake, UP))
print_board(board, move_snake(move_snake(snake, UP), LEFT))

| = | = | = | = | 
|   |   |   | = | 
|   |   | : | = | 

|   | = | = | = | 
|   |   | : | = | 
|   |   | = | = | 

|   |   | = | = | 
|   | : | = | = | 
|   |   | = | = | 



Now, we are going to define a function that, given a path in string format, we are able to print it all

In [7]:
def print_snake_path(board : list, snake : list, path : str):
    """Print the full given path of the snake in the board

    Parameters
    ----------
    board : list
        A list of length 2 with the dimensions of the board.
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    path : str
        A string with the path the snake must go.
    """
    # Print actual snake
    print_board(board, snake)
    
    # Stop condition
    if path == '':
        return
    
    # Move and print the rest of the path
    print_snake_path(board, move_snake(snake, DIRECTIONS_MAP[path[0]]), path[1:])

In [8]:
print_snake_path(board, snake, 'LUL')

| = | = | = | = | 
|   |   |   | = | 
|   |   | : | = | 

|   | = | = | = | 
|   |   |   | = | 
|   | : | = | = | 

|   |   | = | = | 
|   | : |   | = | 
|   | = | = | = | 

|   |   |   | = | 
| : | = |   | = | 
|   | = | = | = | 



To check if the movement performed by the snake is valid, we have to perform two verifications:
- **Self-intersection**: As we can assume that the initial position is correct, in a movement, the only "new" cell occupied by the snake is the new head, so we only have to check if the new head is in a cell already occupied by the body.
- **Out of borders**: The other condition that must be fulfilled is that the snake must be inside the board, that is, that it does not go beyond the edges. As before, in a movement, the only part we have to check is the new head of the snake, as it is the only part that occupy a new cell.

In [9]:
def valid_snake(board : list, snake : list):
    """Check if the snake given is valid in the given board

    Parameters
    ----------
    board : list
        A list of length 2 with the dimensions of the board.
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    
    Returns
    -------
    bool
        True if is valid, False if not
    """
    snakes_head = snake[0]
    snakes_body = snake[1:]
    
    # Self-intersection
    if snakes_head in snakes_body:
        return False
    
    # Inside of board borders
    if 0 <= snakes_head[HORIZONTAL_INDEX] < board[HORIZONTAL_INDEX] and 0 <= snakes_head[VERTICAL_INDEX] < board[VERTICAL_INDEX]:
        return True
    
    # Outsise of board borders
    return False

In [10]:
print_board(board, snake)

print('Snake moves left is valid:', valid_snake(board, move_snake(snake, LEFT)))
print('Snake moves right is not valid (self-intersection):', valid_snake(board, move_snake(snake, DOWN)))
print('Snake moves down is not valid (out of board borders):', valid_snake(board, move_snake(snake, DOWN)))
print('Snake moves up is valid:', valid_snake(board, move_snake(snake, UP)))

| = | = | = | = | 
|   |   |   | = | 
|   |   | : | = | 

Snake moves left is valid: True
Snake moves right is not valid (self-intersection): False
Snake moves down is not valid (out of board borders): False
Snake moves up is valid: True


## Solution

To solve this problem the optimal way we have found is to implement a **[Backtracking algorithm](https://en.wikipedia.org/wiki/Backtracking)** recursively. With this algorithm we are able to go over all the posible solutions and return the correct paths. In this case, we are going to use recursivility to divide the problem in four subproblems (one per each direction). Then, we are going to combine the answers of each problem to solve the actual big problem (get all the possible paths).

The **pseudocode of the function** would be like this:

```
get_snake_paths(board, snake, depth, direction):
    move_snake(snake, direction)
    valid_snake(board, snake)
    
    if no more depth to go:
        return direction # We have a solution here!
    
    get_snake_paths(board, snake, depth-1, LEFT)
    get_snake_paths(board, snake, depth-1, RIGHT)
    get_snake_paths(board, snake, depth-1, DOWN)
    get_snake_paths(board, snake, depth-1, UP)
    
    combine the partial solutions

    add previous move to each partial solution
    
    return all paths of this step in the depth
```

However, **we have to take care of the first step**, where the snake has not move yet. In that case, we will indicate that direction is `None` and it is not necessary to move the snake to any direction, check if its valid (as the base position of the snake is always valid) or add the previous move to the partial solutions.

Knowing all of that, we can build the following function:

### Base solution

In [11]:
def get_snake_paths(board : list, snake : list, depth : int, direction : int = None):
    """Check if the snake given is valid in the given board

    Parameters
    ----------
    board : list
        A list of length 2 with the dimensions of the board.
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    depth : int
        The paths depth
    direction : int
        The direction to move the snake. None indicates that no movement has occur yet. -> 0, 1, 2, 3 = LEFT, RIGHT, DOWN, UP
    
    Returns
    -------
    list
        Lists of possible paths, with the given depth, in string format the snake can go. 
    """
    # Create the snake moved
    if direction is not None:
        snake = move_snake(snake, direction)
    
        # If the snake did an invalid movement, then return empty array
        if not valid_snake(board, snake):
            return []

    # Stop condition, snake has move to the full depth
    if depth == 0:
        return [DIRECTIONS_LETTERS[direction]]
    
    # Get the paths for every direction
    paths_left = get_snake_paths(board, snake, depth-1, direction=LEFT)
    paths_right = get_snake_paths(board, snake, depth-1, direction=RIGHT)
    paths_down = get_snake_paths(board, snake, depth-1, direction=DOWN)
    paths_up = get_snake_paths(board, snake, depth-1, direction=UP)

    # Combine all partial solutions
    paths = paths_left + paths_right + paths_down + paths_up
    
    # Add the previus move (if exists)
    if direction is not None:
        paths = [DIRECTIONS_LETTERS[direction] + path for path in paths]
    
    return paths

In [12]:
print_board(board, snake)
get_snake_paths(board, snake, depth)

| = | = | = | = | 
|   |   |   | = | 
|   |   | : | = | 



['LLU', 'LUL', 'LUR', 'LUU', 'ULL', 'ULD', 'ULU']

### Just returning the number of paths

But, for solve the problem we dont really need the paths, we only need the number of possible paths, so we remove the path creation steps so we optimize the computation time.

In [13]:
def get_n_snake_paths(board : list, snake : list, depth : int, direction : int = None):
    """Check if the snake given is valid in the given board

    Parameters
    ----------
    board : list
        A list of length 2 with the dimensions of the board.
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    depth : int
        The paths depth
    direction : int
        The direction to move the snake. None indicates that no movement has occur yet. -> 0, 1, 2, 3 = LEFT, RIGHT, DOWN, UP
    
    Returns
    -------
    int
        Number of possible paths, with the given depth, the snake can go. 
    """
    # Create the snake moved
    if direction is not None:
        snake = move_snake(snake, direction)
    
        # If the snake did an invalid movement, then return empty array
        if not valid_snake(board, snake):
            return 0

    # Stop condition, snake has move to the full depth. We have found a path!
    if depth == 0:
        return 1
    
    # Get the paths for every direction
    paths_left = get_n_snake_paths(board, snake, depth-1, direction=LEFT)
    paths_right = get_n_snake_paths(board, snake, depth-1, direction=RIGHT)
    paths_down = get_n_snake_paths(board, snake, depth-1, direction=DOWN)
    paths_up = get_n_snake_paths(board, snake, depth-1, direction=UP)

    # Combine all partial solutions
    paths = paths_left + paths_right + paths_down + paths_up
    
    return paths

In [14]:
get_n_snake_paths(board, snake, depth)

7

### Pruning some wrong movements before calculating them

In addition, we can add some pruning to avoid the creation and validation of unnecessary movements.

In [15]:
def get_n_snake_paths_opt(board : list, snake : list, depth : int, direction : int = None):
    """Check if the snake given is valid in the given board

    Parameters
    ----------
    board : list
        A list of length 2 with the dimensions of the board.
    snake : list
        A list of cells (list of two ints) indicating the cells where the snake is. First cell is head and last tail.
    depth : int
        The paths depth
    direction : int
        The direction to move the snake. None indicates that no movement has occur yet. -> 0, 1, 2, 3 = LEFT, RIGHT, DOWN, UP
    
    Returns
    -------
    int
        Number of possible paths, with the given depth, the snake can go. 
    """
    # Create the snake moved
    if direction is not None:
        snake = move_snake(snake, direction)
    
        # If the snake did an invalid movement, then return empty array
        if not valid_snake(board, snake):
            return 0

    # Stop condition, snake has move to the full depth. We have found a path!
    if depth == 0:
        return 1
    
    # Get the paths for every direction
    paths_left = get_n_snake_paths_opt(board, snake, depth-1, direction=LEFT) if (direction is None or direction != RIGHT) and snake[0][HORIZONTAL_INDEX] > 0 else 0
    paths_right = get_n_snake_paths_opt(board, snake, depth-1, direction=RIGHT) if (direction is None or direction != LEFT) and snake[0][HORIZONTAL_INDEX] < (board[HORIZONTAL_INDEX]-1) else 0
    paths_down = get_n_snake_paths_opt(board, snake, depth-1, direction=DOWN) if (direction is None or direction != UP) and snake[0][VERTICAL_INDEX] < (board[VERTICAL_INDEX]-1) else 0
    paths_up = get_n_snake_paths_opt(board, snake, depth-1, direction=UP) if (direction is None or direction != DOWN) and snake[0][VERTICAL_INDEX] > 0  else 0

    # Combine all partial solutions
    paths = paths_left + paths_right + paths_down + paths_up
    
    return paths

In [16]:
get_n_snake_paths_opt(board, snake, depth)

7

### Comparing solution times

Finally, we are going to display the times of each function for a board of 10x10, the same snake and depth 19 (cannot run with depth=20 because kernel dies in jupyter):

In [17]:
board = [10, 10]
snake = [
    [2, 2],
    [3, 2],
    [3, 1],
    [3, 0],
    [2, 0],
    [1, 0],
    [0, 0]
]
depth = 19

%time _ = get_snake_paths(board, snake, depth)
%time _ = get_n_snake_paths(board, snake, depth)
%time _ = get_n_snake_paths_opt(board, snake, depth)

CPU times: user 2min 22s, sys: 2.87 s, total: 2min 25s
Wall time: 2min 25s
CPU times: user 1min 17s, sys: 39.9 ms, total: 1min 17s
Wall time: 1min 17s
CPU times: user 1min 2s, sys: 0 ns, total: 1min 2s
Wall time: 1min 2s


### Other ways to optimize the problem

A possible optimitation we could implement if we want even more efficiency is the use of parallelization, since this problem can be greatly exploited by the use of recursive functions.